<a href="https://colab.research.google.com/github/alisaghaffi/big-data-challenge/blob/main/BigData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
# import dependencies
import os
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 2s (108 kB/s)
Reading package lis

In [59]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [105]:
 # Read in data from S3 Buckets
from pyspark import SparkFiles

url = "https://alibigdata.s3.us-west-1.amazonaws.com/Reviews.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("Reviews.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
df.show(10)

+---+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+
| Id| ProductId|        UserId|         ProfileName|HelpfulnessNumerator|HelpfulnessDenominator|Score|      Time|             Summary|                Text|
+---+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+
|  1|B001E4KFG0|A3SGXH7AUHU8GW|          delmartian|                   1|                     1|    5|1303862400|Good Quality Dog ...|I have bought sev...|
|  2|B00813GRG4|A1D87F6ZCVE5NK|              dll pa|                   0|                     0|    1|1346976000|   Not as Advertised|"Product arrived ...|
|  3|B000LQOCH0| ABXLMWJIXXAIN|"Natalia Corres "...|                   1|                     1|    4|1219017600|"""Delight"" says...|"This is a confec...|
|  4|B000UA0QIQ|A395BORC6FGVXV|                Karl|            

In [106]:
#   * Count the number of records (rows) in the dataset.
df.dropna()
df.count()

568454

In [107]:

df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- UserId: string (nullable = true)
 |-- ProfileName: string (nullable = true)
 |-- HelpfulnessNumerator: string (nullable = true)
 |-- HelpfulnessDenominator: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Text: string (nullable = true)



In [54]:
#   * **Transform** the dataset to fit the tables in the [schema file](../Resources/schema.sql). 
# Be sure the DataFrames match in data type and in column name.
#schema_df = df.select(["Id", "ProductId", "UserId", "ProfileName","HelpfulnessNumerator","HelpfulnessDenominator", "Score","Time", "Summary" ,"Text"])
#schema_df = schema_df.dropDuplicates(["ProductId"])
#schema_df.head()

Row(Id=512509, ProductId='B00008DFTP', UserId='A1B6JV6UGVLMTT', ProfileName='Jebby', HelpfulnessNumerator='2', HelpfulnessDenominator='2', Score='5', Time='1319241600', Summary='Great Value', Text='My dog loves this food.  He was previously eating Iams, which I did like, but it became hard to find.  So, i switched to this.  I think it makes his coat nicer than the Iams.  This is a tremendous value for this brand.  It is pricier in the store.')

In [110]:
# Changing datatypes
# Source Link: https://stackoverflow.com/questions/32284620/how-to-change-a-dataframe-column-from-string-type-to-double-type-in-pyspark

from pyspark.sql.types import TimestampType, IntegerType

df = df.withColumn("customer_id", df["UserId"].cast(IntegerType()))\
       .withColumn("product_parent", df["ProductId"].cast(IntegerType()))\
       .withColumn("review_date", df["Time"].cast(TimestampType()))\
       .withColumn("star_rating", df["Score"].cast(IntegerType()))\
       .withColumn("helpful_votes", df["HelpfulnessNumerator"].cast(IntegerType()))\
      .withColumn("total_votes", df["HelpfulnessDenominator"].cast(IntegerType()))\

# Print schema
df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- UserId: string (nullable = true)
 |-- ProfileName: string (nullable = true)
 |-- HelpfulnessNumerator: string (nullable = true)
 |-- HelpfulnessDenominator: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Text: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: timestamp (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)



In [81]:
# Create review dataframe to match review_id_table table
review_id_df = df.select(["Id","ProfileName","ProductId", "helpful_votes","Time"])
review_id_df.show()

+---+--------------------+----------+-------------+----------+
| Id|         ProfileName| ProductId|helpful_votes|      Time|
+---+--------------------+----------+-------------+----------+
|  1|          delmartian|B001E4KFG0|            1|1303862400|
|  2|              dll pa|B00813GRG4|            0|1346976000|
|  3|"Natalia Corres "...|B000LQOCH0|            1|1219017600|
|  4|                Karl|B000UA0QIQ|            3|1307923200|
|  5|"Michael D. Bigha...|B006K2ZZ7K|            0|1350777600|
|  6|      Twoapennything|B006K2ZZ7K|            0|1342051200|
|  7|   David C. Sullivan|B006K2ZZ7K|            0|1340150400|
|  8|  Pamela G. Williams|B006K2ZZ7K|            0|1336003200|
|  9|            R. James|B000E7L2R4|            1|1322006400|
| 10|       Carol A. Reed|B00171APVA|            0|1351209600|
| 11|        Canadian Fan|B0001PB9FE|            1|1107820800|
| 12|"A Poeng ""Sparky...|B0009XLVG0|            4|1282867200|
| 13|                  LT|B0009XLVG0|            1|1339

In [82]:
# Print schema
review_id_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- ProfileName: string (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- Time: string (nullable = true)



In [83]:
# Create customer dataframe to match customers table
from pyspark.sql.functions import desc

customers_df = df.groupby("ProfileName").agg({"ProductId":"count"})
customers_df = customers_df.orderBy(desc("count(ProductId)"))
customers_df = customers_df.withColumnRenamed("count(ProductId)", "customer_count") 
customers_df.show()

+--------------------+--------------+
|         ProfileName|customer_count|
+--------------------+--------------+
|"C. F. Hill ""CFH"""|           451|
|"O. Brown ""Ms. O...|           421|
|       Gary Peterson|           389|
|"Rebecca of Amazo...|           365|
|               Chris|           363|
|               Linda|           290|
|                John|           261|
|                Mike|           260|
|                  c2|           256|
|               Laura|           253|
|               Sarah|           244|
|           Stephanie|           240|
|               Karen|           239|
|                Lisa|           237|
|             Jessica|           236|
|                 Jen|           232|
|               Steve|           225|
|               David|           216|
|           Anonymous|           213|
|                Mary|           208|
+--------------------+--------------+
only showing top 20 rows



In [87]:
# Create product dataframe to match products table(only unique values)
products_df = df.select(["ProductId","Summary"])
products_df = products_df.dropDuplicates()
products_df.show()

+----------+--------------------+
| ProductId|             Summary|
+----------+--------------------+
|B001ELL6O8|Arrowhead Mills w...|
|B000G6RYNE|The only thing I ...|
|B002XG21MO|The Puppy Dogs Lo...|
|B0028SWACS|           Delicious|
|B000WNJ73Q|    Rich J Wyzykoski|
|B000ODRY9I|         Its GOOD!!!|
|B0000VLH8S|        shining star|
|B009UOFU20|weak coffee not g...|
|B001RVFDOO|Better than expected|
|B004ET9OIW|Why I received on...|
|B0000CGFV4|Nice, Big Pieces ...|
|B000J2DQ46|              MMMMMM|
|B0085G4ACA|The answer to all...|
|B000LKU03G|Awesome with some...|
|B005K4Q1VI| Sugar AND Sucralose|
|B005K4Q1VI|          Good drink|
|B005K4Q1VI|Tasty and Good fo...|
|B005K4Q1VI|Great as a creame...|
|B001QEF4XC|Excellent, health...|
|B0045IK1CK|Taste like muddy ...|
+----------+--------------------+
only showing top 20 rows



In [88]:
# Print schema
review_id_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- ProfileName: string (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- Time: string (nullable = true)



In [93]:

# Create product dataframe to match products table(only unique values)
products_df = df.select(["ProductId","time"])
products_df = products_df.dropDuplicates()
products_df.show()

+----------+----------+
| ProductId|      time|
+----------+----------+
|B001GVISJM|1304899200|
|B000ITVLE2|1348099200|
|B001EO5ZMO|1330819200|
|B001UJEN6C|1322697600|
|B002GUWBMC|1328400000|
|B000VKYKTG|1259539200|
|B000JWEEB0|1206748800|
|B001CWZXIY|1310860800|
|B001RVFDOO|1261094400|
|B001RVFDOO|1336608000|
|B0007T3V82|1218672000|
|B0016D2MY2|1272153600|
|B0001VWE0C|1127606400|
|B0089SPDUW|1328918400|
|B002JX7GVM|1264550400|
|B000B9SFP2|1311206400|
|B005K4Q1VI|1331856000|
|B001QEF4XC|1344556800|
|B001FZR732|1293321600|
|B004391DK0|1335657600|
+----------+----------+
only showing top 20 rows



In [97]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<insert endpoint>:5432/bigdata"
config = {"user":"posstgres",
          #"password": "<insert password>",
          "password": "posstgres",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to review_id_table table in RDS

review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write DataFrame to products table in RDS

products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to customers table in RDS

customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to vine_table table in RDS

vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)